In [91]:
import mysql.connector as mysql
from sqlalchemy import create_engine
import pandas as pd

In [92]:
connection = mysql.connect(user='root', 
                           password='softzino',
                           host='127.0.0.1',
                           database='new_schema')

In [93]:
connection.is_connected()

True

In [94]:
df = pd.read_csv('train_output.csv')



In [95]:
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')

In [80]:
df = df.fillna('NULL')

In [96]:
df.head(3)

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,Country,City,State,Postal_Code,Region,Product_ID,Category,Sub_Category,Product_Name,Sales
0,1,CA_2017_152156,2017-11-08 00:00:00,2017-11-11 00:00:00,Second Class,CG_12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96
1,2,CA_2017_152156,2017-11-08 00:00:00,2017-11-11 00:00:00,Second Class,CG_12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94
2,3,CA_2017_138688,2017-06-12 00:00:00,2017-06-16 00:00:00,Second Class,DV_13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62


In [97]:
df.dtypes

Row_ID             int64
Order_ID          object
Order_Date        object
Ship_Date         object
Ship_Mode         object
Customer_ID       object
Customer_Name     object
Segment           object
Country           object
City              object
State             object
Postal_Code      float64
Region            object
Product_ID        object
Category          object
Sub_Category      object
Product_Name      object
Sales            float64
dtype: object

In [98]:
import re

In [99]:
create_col = []

In [100]:
for col, type in df.dtypes.items():
    if re.search('date', col) or re.search('Date', col) :
        create_col.append({
            'column_name': str(col),
            'column_type': 'datetime'
        })
    elif type == 'int64':
        create_col.append({
            'column_name': str(col),
            'column_type': 'numeric'
        })
    elif type == 'float64':
        create_col.append({
            'column_name': str(col),
            'column_type': 'numeric'
        })
    elif type == 'object':
        create_col.append({
            'column_name': str(col),
            'column_type': 'varchar(255)'
        })

In [101]:
create_col

[{'column_name': 'Row_ID', 'column_type': 'numeric'},
 {'column_name': 'Order_ID', 'column_type': 'varchar(255)'},
 {'column_name': 'Order_Date', 'column_type': 'datetime'},
 {'column_name': 'Ship_Date', 'column_type': 'datetime'},
 {'column_name': 'Ship_Mode', 'column_type': 'varchar(255)'},
 {'column_name': 'Customer_ID', 'column_type': 'varchar(255)'},
 {'column_name': 'Customer_Name', 'column_type': 'varchar(255)'},
 {'column_name': 'Segment', 'column_type': 'varchar(255)'},
 {'column_name': 'Country', 'column_type': 'varchar(255)'},
 {'column_name': 'City', 'column_type': 'varchar(255)'},
 {'column_name': 'State', 'column_type': 'varchar(255)'},
 {'column_name': 'Postal_Code', 'column_type': 'numeric'},
 {'column_name': 'Region', 'column_type': 'varchar(255)'},
 {'column_name': 'Product_ID', 'column_type': 'varchar(255)'},
 {'column_name': 'Category', 'column_type': 'varchar(255)'},
 {'column_name': 'Sub_Category', 'column_type': 'varchar(255)'},
 {'column_name': 'Product_Name', '

In [102]:
def storing_to_database(table_name, df):

        cursor = connection.cursor()
        
        text_final = ""
    
        for x in create_col:
            if x['column_name'] == "Sales":
                text_final += f"{x['column_name']} {x['column_type']}"
            else:
               text_final += f"{x['column_name']} {x['column_type']}, "  

        print(text_final)
            
        query = f"CREATE TABLE IF NOT EXISTS {table_name} ({text_final})"
    
        print(query)
    
        cursor.execute(query)

        text_final = ""

        print(text_final)
    
        print(df)
                       
        
        print(f"Successfuly create table {table_name}")

In [103]:
df = pd.read_csv('train_output.csv')

storing_to_database("table_28", df)  

Row_ID numeric, Order_ID varchar(255), Order_Date datetime, Ship_Date datetime, Ship_Mode varchar(255), Customer_ID varchar(255), Customer_Name varchar(255), Segment varchar(255), Country varchar(255), City varchar(255), State varchar(255), Postal_Code numeric, Region varchar(255), Product_ID varchar(255), Category varchar(255), Sub_Category varchar(255), Product_Name varchar(255), Sales numeric
CREATE TABLE IF NOT EXISTS table_28 (Row_ID numeric, Order_ID varchar(255), Order_Date datetime, Ship_Date datetime, Ship_Mode varchar(255), Customer_ID varchar(255), Customer_Name varchar(255), Segment varchar(255), Country varchar(255), City varchar(255), State varchar(255), Postal_Code numeric, Region varchar(255), Product_ID varchar(255), Category varchar(255), Sub_Category varchar(255), Product_Name varchar(255), Sales numeric)

      Row ID        Order ID           Order Date            Ship Date  \
0          1  CA_2017_152156  2017-11-08 00:00:00  2017-11-11 00:00:00   
1          2  C

In [109]:
df.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub_Category',
       'Product Name', 'Sales'],
      dtype='object')

In [113]:
from tqdm import tqdm

In [114]:
m = df.values

for i in tqdm(range(len(df))):
    
    query_text = "("
    for j in range(18):
        
        if create_col[j]['column_type'] == "varchar(255)" or  create_col[j]['column_type'] == "datetime":
            query_text += "'"
            if escape_val:
                escape_val = connection._cmysql.escape_string(str(m[i][j]))
                escape_val = escape_val.decode('utf-8')
            else:
                escape_val = 0
            query_text += f"{escape_val}"
            query_text += "', "
        else:
            if j !=17:
                query_text += str(m[i][j])
                query_text += ","
            else:
                query_text += str(m[i][j])
                
                      
    query_text += ")"
    
    
    main_query = f"INSERT INTO table_28 VALUES {query_text}"
    
    cursor = connection.cursor()
    cursor.execute(main_query)

connection.commit() 

100%|█████████████████████████████████████| 9800/9800 [00:04<00:00, 2228.34it/s]


In [ ]:
query = 'INSERT INTO {tablename} VALUES (%s, )'